Support Vector Regression 에서 이용했던 시계열 형태의 데이터에 대한 회귀 모델을 위한 random forest 를 학습해 봅니다. Scatter plot 에서 볼 수 있듯이 어떠한 패턴이 존재하지 않는 데이터입니다.

In [1]:
from bokeh.io import save
from bokeh.plotting import output_notebook, show
from soydata.data.regression import make_randomwalk_timeseries_data
from soydata.visualize import scatterplot

output_notebook()

x, y, y_true = make_randomwalk_timeseries_data(n_repeats=1, noise=0.2, std=1, seed=0)
p = scatterplot(x, y, size=1, height=200)

Loading BokehJS ...

그 경향을 확실히 살펴보기 위해 강제로 under fitting 이 나도록 max_depth 를 제한합니다. Estimators 의 개수도 5개로 적기 때문에 under fitting 이 일어납니다. 또한 regression 용 random forest 에서는 oob score 를 측정할 수 없습니다. 

각각의 base estimator 가 over fitting 이 일어날수록 random forest 의 예측력도 좋아집니다.

In [2]:
import numpy as np
from bokeh.layouts import gridplot
from sklearn.ensemble import RandomForestRegressor

grids = []

for max_depth in [5, 10, 50]:
    rf = RandomForestRegressor(n_estimators=5, bootstrap=True, oob_score=False, max_depth=max_depth)
    X = x.reshape(-1,1)
    rf.fit(X, y)
    y_pred = rf.predict(X)

    title = f'RF (#DT=10), max-depth={max_depth}'
    p = scatterplot(x, y, size=1, height=300, width=800, color='grey', title=title, show_inline=False)
    p = scatterplot(x, y_pred, size=1, p=p, show_inline=False)
    grids.append([p])

gp_md = gridplot(grids)
show(gp_md)

각각의 base estimators 는 `RandomForestRegressor.estimators_` 에 들어있습니다. 이들 각각의 prediction value 를 계산합니다. numpy.vstack 을 이용하여 여러 개의 column vectors 를 행렬 형태로 변환합니다. Rows 가 각 base estimator 별 예측값이기 때문에 transpose 를 한 번 합니다. 이 값에서 std 를 계산하면 5 개의 예측값의 표준편차를 계산할 수 있습니다. Base estimator 끼리 서로 다른 값을 예측하는 구간에서는 이 값이 큽니다.

In [3]:
rf = RandomForestRegressor(n_estimators=5, bootstrap=True, oob_score=False, max_depth=5)
rf.fit(X, y)
y_preds = np.vstack([base.predict(X) for base in rf.estimators_])
print(f'shape of y_preds = {y_preds.shape}')

y_preds = y_preds.T
y_std = y_preds.std(axis=1)
print(f'shape of y_preds = {y_preds.shape}')
print(f'shape of y_std = {y_std.shape}')

shape of y_preds = (5, 500)
shape of y_preds = (500, 5)
shape of y_std = (500,)


(prediction - 2 * std, prediction + 2 * std) 를 상한과 하한으로 설정하여 confidence interval 을 설정합니다. Under fitting 된 decision trees 때문에 std 가 큰 구간들이 존재합니다.

In [4]:
from bokeh.palettes import Reds9
from soydata.visualize import lineplot

title = f'RF (#DT=5), max-depth=5 with C.I.'
pairs = [(i, i+1) for i in range(x.shape[0]-1)]
p_regci0 = scatterplot(x, y, size=1, height=300, width=800, title=title, show_inline=False)
p_regci0 = scatterplot(x, y_pred, size=1, color=Reds9[0], p=p_regci0, show_inline=False)
p_regci0 = lineplot(x, y_pred - 2 * y_std, pairs, line_width=0.5, alpha=0.5, color=Reds9[5], p=p_regci0, show_inline=False)
p_regci0 = lineplot(x, y_pred + 2 * y_std, pairs, line_width=0.5, alpha=0.5, color=Reds9[5], p=p_regci0, show_inline=True)

이번에는 estimators 의 개수와 max depth 를 크게 설정하여 각 base estimators 가 over fitting 이 발생하도록 유도한 뒤, confidence interval 과 함께 회귀예측을 수행합니다. 앞선 결과보다 confidence interval 의 간격이 줄어들었습니다.

In [5]:
rf = RandomForestRegressor(n_estimators=20, bootstrap=True, oob_score=False, max_depth=20)
rf.fit(X, y)
y_preds = np.vstack([base.predict(X) for base in rf.estimators_])
y_preds = y_preds.T
y_std = y_preds.std(axis=1)

title = f'RF (#DT=20), max-depth=20 with C.I.'
pairs = [(i, i+1) for i in range(x.shape[0]-1)]
p_regci1 = scatterplot(x, y, size=1, height=300, width=800, title=title, show_inline=False)
p_regci1 = scatterplot(x, y_pred, size=1, color=Reds9[0], p=p_regci1, show_inline=False)
p_regci1 = lineplot(x, y_pred - 2 * y_std, pairs, line_width=0.5, alpha=0.5, color=Reds9[5], p=p_regci1, show_inline=False)
p_regci1 = lineplot(x, y_pred + 2 * y_std, pairs, line_width=0.5, alpha=0.5, color=Reds9[5], p=p_regci1, show_inline=False)

gp_regci = gridplot([[p_regci0], [p_regci1]])
show(gp_regci)

In [7]:
# save(gp_md, './figures/random_forest_regression.html')
# save(gp_regci, './figures/random_forest_regression_with_ci.html')